# Classification
**Objective**: Predict if an earthquake causes a tsunami

## Data Loading

## Splitting Data

## Preprocessing

## LazyPredict Benchmark

## Hyperparameters Tuning (Top 3 model)

## Model Comparison

## Best Model Evaluation